## 🎯 Pregunta de negocio  
**¿Cómo puede Mercado Libre identificar a los sellers más relevantes y segmentarlos para diseñar estrategias comerciales personalizadas?**

🧠 **Hipótesis inicial:**  
Los sellers con **alto stock**, **muchas publicaciones** y **cero productos reacondicionados** tienden a ser más relevantes para el negocio.

In [1]:
import sys
from IPython.display import Markdown, display
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
sys.path.append("C:/Users/dario/Documents/worksapce/sellers_analitycs_meli")

from meli_insight_engine.llm.prompts.templates import BASIC_RECOMMENDATION_PROMPT_JSON, BASIC_RECOMMENDATION_PROMPT_HIPOTESIS
from meli_insight_engine.llm.agents.agent_template import TemplateAgent
from meli_insight_engine.core import (
    conectar_duckdb,
    registrar_csvs_como_vistas,
    verificar_vistas,
    analisis_inicial_completo,
    guardar_resultados_como_txt
)
import joblib, os


# 🧠 Descubrimiento inicial del dataset con MAD-G

Este notebook realiza una **revisión estructural y exploración inicial** del archivo `df_challenge.csv`, en el contexto del challenge de segmentación de sellers de Mercado Libre.

📍 **Objetivo general:**
Comprender la composición y calidad del dataset para diseñar una solución de clusterización efectiva y alineada con necesidades comerciales.

📌 **Lo que haremos en esta etapa:**
- Analizar la estructura del dataset (columnas, tipos de datos, duplicados, nulos, constantes).
- Detectar variables relevantes para caracterizar a los sellers.
- Evaluar posibles problemas de calidad de datos que deban corregirse o imputarse.
- Generar **hipótesis de negocio iniciales** con ayuda de un modelo de lenguaje (LLM), a partir de la observación de los datos.
  
🧩 **¿Qué estamos construyendo?**
Una herramienta analítica que combina la exploración de datos tradicional con un asistente de IA (LLM), capaz de:
- Proponer agrupaciones lógicas de vendedores.
- Sugerir estrategias de segmentación.
- Enriquecer el entendimiento de los datos con razonamiento automatizado.

Esta exploración será la base para la ingeniería de features y el modelado posterior.


## ⚙️ Configuración del entorno y carga de datos

### Analisis de datos para levantar hipotesis usando LLM 

In [2]:

# Crear conexión a DuckDB
con = conectar_duckdb()

# Ruta a tu carpeta con archivos .csv de prueba (asegúrate de que exista y tenga al menos un .csv)
folder_path = "../data/"
# Registrar vistas
registrar_csvs_como_vistas(con, folder_path)


In [3]:


# Verificar vistas
print("✅ Vistas registradas:")
print(verificar_vistas(con))

# Reemplaza con el nombre real del archivo sin .csv y sin guiones (los guiones se vuelven guiones bajos)
nombre_tabla = "data.df_challenge_meli"# Ejemplo: si el archivo se llama "ventas-julio.csv", usa "data.ventas_julio"

resultados = analisis_inicial_completo(con, "data.df_challenge_meli")
guardar_resultados_como_txt(resultados)


✅ Vistas registradas:
                        table_name
0     data.cluster_stats_empiricas
1           data.df_challenge_meli
2  data.df_challenge_meli_clusters
3    data.df_challenge_meli_limpio
4                data.pca_loadings


C:\Users/dario/Documents/worksapce/sellers_analitycs_meli\meli_insight_engine\core\inspector.py:359: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fechas_parseadas = pd.to_datetime(df[col], errors='coerce')
C:\Users/dario/Documents/worksapce/sellers_analitycs_meli\meli_insight_engine\core\inspector.py:359: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fechas_parseadas = pd.to_datetime(df[col], errors='coerce')
C:\Users/dario/Documents/worksapce/sellers_analitycs_meli\meli_insight_engine\core\inspector.py:359: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fechas_parseadas = 

✅ Resultados guardados en: ../data/outputs_prompts/inspector_stats.txt


In [4]:

# Ruta del archivo .txt
ruta_txt = "../data/outputs_prompts/inspector_stats.txt"

# Leer contenido del archivo
with open(ruta_txt, "r", encoding="utf-8") as f:
    texto = f.read()

    # Instanciar el agente con el template y el texto leído
    agent = TemplateAgent(prompt_template=BASIC_RECOMMENDATION_PROMPT_JSON,template_name= "BASIC_RECOMMENDATION_PROMPT_JSON" )

# Ejecutar
respuesta = agent.run(input_path=ruta_txt)

# Mostrar respuesta
print(respuesta)

c:\Users\dario\anaconda3\envs\meli_challenge\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\dario\anaconda3\envs\meli_challenge\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


../data/outputs_prompts\BASIC_RECOMMENDATION_PROMPT_JSON.txt


In [5]:


# Ruta del archivo .txt
ruta_txt = "../data/outputs_prompts/BASIC_RECOMMENDATION_PROMPT_JSON.txt"

# Leer contenido del archivo
with open(ruta_txt, "r", encoding="utf-8") as f:
    texto = f.read()

    # Instanciar el agente con el template y el texto leído
    agent = TemplateAgent(prompt_template=BASIC_RECOMMENDATION_PROMPT_HIPOTESIS,template_name= "BASIC_RECOMMENDATION_PROMPT_HIPOTESIS" )

# Ejecutar
respuesta = agent.run(input_path=ruta_txt)

# Mostrar respuesta
print(respuesta)

../data/outputs_prompts\BASIC_RECOMMENDATION_PROMPT_HIPOTESIS.txt


#### resultados de MDA-G (models for data Analysis)

In [6]:

# Ruta al archivo externo (por ejemplo, en una carpeta docs o en el root)
ruta_readme = "../data/outputs_prompts/BASIC_RECOMMENDATION_PROMPT_HIPOTESIS.txt"

# Leer y mostrar
with open(ruta_readme, "r", encoding="utf-8") as archivo:
    contenido = archivo.read()

display(Markdown(contenido))


### **Reporte de Comprensión Inicial del Dataset**

---

#### **1. Estructura Básica**
- **Total registros**: 185,250  
- **Total variables**: 14  
- **Variables clave identificadas**:  
  - **[Numérica Continua] `price`**: Precio de venta con valores extremos (hasta 4.7B) y media en 37,015.  
  - **[Numérica Discreta] `stock`**: Inventario altamente sesgado (moda = 1, max = 99,999).  
  - **[Categórica Ordinal] `seller_reputation`**: Jerarquía de reputación (newbie → platinum) con dominio de vendedores platinum (37.6%).  
  - **[Categórica Nominal] `logistic_type`**: Logística dominada por "XD" (63%, propio de MercadoLibre).  
  - **[Booleana] `is_refurbished`**: Desbalance extremo (0.4% refurbished).  

---

#### **2. Hallazgos de Calidad**  
- **`price`**:  
  - **Tipo de problema**: Outliers extremos (max = 4.7B vs. media = 37K).  
  - **Severidad**: Alta (distorsiona estadísticas).  
  - **Acción**: Winsorización al 99.9% + transformación logarítmica.  

- **`regular_price`**:  
  - **Tipo de problema**: 73% nulos + inconsistencia con `price` (valores frecuentes repetidos).  
  - **Severidad**: Media-Alta (afecta análisis de descuentos).  
  - **Acción**: Imputar con `price` * 1.2 (default) o eliminar.  

- **`seller_reputation`**:  
  - **Tipo de problema**: 1.3% nulos + jerarquía ordinal no codificada.  
  - **Severidad**: Media.  
  - **Acción**: Codificación ordinal (newbie=1, platinum=5).  

- **`is_refurbished`**:  
  - **Tipo de problema**: Desbalance (735 vs. 184K).  
  - **Severidad**: Alta para modelos predictivos.  
  - **Acción**: Oversampling (target ratio = 10%).  

- **`tim_day`**:  
  - **Tipo de problema**: Constante (único valor: 2024-08-01).  
  - **Severidad**: Baja (sin impacto analítico).  
  - **Acción**: Eliminar.  

---

#### **3. Relaciones Potenciales**  
- **`seller_reputation` ↔ `price`**: ¿Vendedores premium fijan precios más altos? Validar con ANOVA o boxplots por tier.  
- **`logistic_type` ↔ `stock`**: ¿Logística "XD" tiene mejor gestión de inventario? Analizar con medianas agrupadas.  
- **`category_id` ↔ `price`**: ¿Categorías específicas tienen precios más altos? Heatmap de precios medios por categoría.  

---

#### **4. Recomendaciones para EDA**  
- **Grupo 1 (Precios y Reputación)**:  
  - Variables: `price`, `regular_price`, `seller_reputation`.  
  - Objetivo: Identificar estrategias de pricing por reputación.  
  - Transformación: Winsorizar `price` + codificar ordinalmente reputación.  

- **Grupo 2 (Inventario y Logística)**:  
  - Variables: `stock`, `logistic_type`, `categoria`.  
  - Objetivo: Explorar eficiencia logística por categoría.  
  - Transformación: Agrupar categorías minoritarias.  

---

#### **5. Advertencias Clave**  
- **Limitación 1**: Outliers en `price` invalidan estadísticas descriptivas actuales.  
  - **Impacto**: Cualquier modelo sin tratamiento será sesgado.  
- **Limitación 2**: Alta cardinalidad en `titulo` (174K únicos).  
  - **Acción**: Usar NLP básico (ej: longitud de texto) o excluir.  

---

#### **6. Conclusión Ejecutiva**  
- **Calidad del Dataset**: **Aceptable con deficiencias críticas**.  
  - **Fortalezas**: Jerarquías claras (reputación), cobertura amplia (185K registros).  
  - **Debilidades**: Outliers extremos, nulos en `regular_price`, desbalances.  
- **Variables Prometedoras**:  
  - `seller_reputation` y `price` para segmentación.  
  - `logistic_type` para análisis operacional.  
- **Riesgos Principales**:  
  - Outliers no tratados invalidarán modelos.  
  - Desbalance en `is_refurbished` requiere técnicas especializadas.  
- **Valor Potencial**:  
  - Con transformaciones, el dataset permite análisis de pricing, eficiencia logística y reputación de vendedores.  

--- 

**Nota Final**: Priorizar limpieza de outliers y codificación ordinal antes de EDA avanzado.

# Fetaured data pre segmentacion

In [8]:

# 1. Cargar el dataset original
df = pd.read_csv('../data/df_challenge_meli.csv')

# 2. Feature engineering inicial
df['discount_pct'] = np.where(
    df['regular_price'].isna() | (df['regular_price'] == 0),
    0,
    (df['regular_price'] - df['price']) / df['regular_price']
)

df['title_len'] = df['titulo'].astype(str).str.len()

rep_map = {
    "green_platinum": 5, "green_gold": 4, "green_silver": 4,
    "green": 3, "light_green": 3, "yellow": 2,
    "orange": 1, "red": 1, "newbie": 0
}
df['rep_score'] = df['seller_reputation'].map(rep_map).fillna(0)

# 3. Agregación a nivel seller
seller = (
    df.groupby('seller_nickname')
      .agg(
          num_publicaciones=('titulo', 'size'),
          log_stock_avg=('stock', lambda s: np.log1p(s).mean()),
          log_price_avg=('price', lambda s: np.log1p(s).mean()),
          porc_descuento=('discount_pct', 'mean'),
          categorias_distintas=('category_id', 'nunique'),
          proporcion_usados=('condition', lambda s: (s == 'used').mean()),
          proporcion_refurb=('is_refurbished', 'mean'), #validar el porque la imputacion de la media 
          rep_score=('rep_score', 'mean'),
          titulo_length_avg=('title_len', 'mean')
      )
      .reset_index()
)

# 4. Definir pipeline (imputación + escalado robusto)
num_cols = seller.columns.difference(['seller_nickname'])
X = seller[num_cols]

pre_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Aplicar las transformaciones
X_scaled = pre_pipe.fit_transform(X)

os.makedirs("models", exist_ok=True)
joblib.dump(pre_pipe, "../models/pre_pipe.pkl")
print("✅ Pipeline guardado en models/pre_pipe.pkl")


# 5. Persistir resultados en un DataFrame limpio
df_final = pd.DataFrame(X_scaled, columns=num_cols)
df_final.insert(0, 'seller_nickname', seller['seller_nickname'])

# Guardar dataset limpio y transformado
df_final.to_csv('../data/df_challenge_meli_limpio.csv', index=False)

# Confirmación visual
print(df_final.head())


✅ Pipeline guardado en models/pre_pipe.pkl
  seller_nickname  categorias_distintas  log_price_avg  log_stock_avg  \
0      000631669c                   0.0       0.179097       0.199550   
1      0007153bca                   0.0      -0.251401       0.546386   
2      000bee3c3b                   0.0      -0.394075      -0.675326   
3      000df2bd02                   0.0       0.590278      -0.021600   
4      000e27cea2                   1.0      -0.208310      -0.181308   

   num_publicaciones  porc_descuento  proporcion_refurb  proporcion_usados  \
0                0.0             0.0                0.0                0.0   
1                0.5             0.0                0.0                0.0   
2                0.5             0.0                0.0                0.0   
3                0.0             0.0                0.0                1.0   
4                0.5             0.0                0.0                0.0   

   rep_score  titulo_length_avg  
0  -1.000000   